In [1]:
!pip install ultralytics
import os
import cv2
import shutil
import numpy as np
from tensorflow.keras.models import load_model
from ultralytics import YOLO
from os.path import isfile

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/57/2e/9d58b392f3cb41f75e1f8e274e528c48c46e47caf581e3e34147780c26dc/ultralytics-8.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for hub-sdk>=0.0.2 from https://files.pythonhosted.org/packages/2f/e0/b3db53becb3c0fa6ad9ddf4d047f4b7dbe85987594bcdd4d1b247f951522/hub_sdk-0.0.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.2/699.2 kB 13.1 MB/s eta 0:00:00a 0:00:01


In [2]:
def video2framesarray(videoinput):
    cap = cv2.VideoCapture(videoinput)
    frame_number = 0
    frame_array = []
    fps = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_array.append(frame)
        frame_number += 1

    cap.release()
    return frame_array

In [3]:
def save_frames_as_images(source_images):
    output_folder = 'temp_images'
    os.makedirs(output_folder, exist_ok=True)

    # Save the images from the array to the folder
    for i, image in enumerate(source_images):
        image_filename = os.path.join(output_folder, f'image_{i}.jpg')
        cv2.imwrite(image_filename, image)

    return output_folder


In [10]:
import os
import cv2
import shutil
from ultralytics import YOLO

def run_yolo(input_folder):
    model = YOLO('yolov8n.pt')
    model = YOLO('yolov8n-pose.pt')# Pretrained YOLOv8n model
    model(source= input_folder, save=True, project='Project' )

    #!yolo task=detect mode=predict model=yolov8n.pt source="temp_images"
    output_images = []
  
    result_files = os.listdir(os.path.join('/kaggle/working/Project/predict'))

    sorted_result_files = sorted(result_files, key=lambda x: int(os.path.splitext(x)[0].split('_')[1]))
    
    # Load the sorted image files into an array
    output_images = [cv2.imread(os.path.join('/kaggle/working/Project/predict', filename)) for filename in result_files]
    return output_images

In [11]:
def detect_foul(image_path, foul_model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize image to match your model's input size
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = foul_model.predict(img)
    if prediction >= 0.5:
        return "foul"
    else:
        return "not-foul"

In [12]:
def process_images(foul_model):
    foul_counter = 0
    output_folder_foul = 'foul_images'
    output_folder_non_foul = 'non_foul_images'
    os.makedirs(output_folder_foul, exist_ok=True)
    os.makedirs(output_folder_non_foul, exist_ok=True)
    
    result_files = os.listdir(os.path.join('/kaggle/working/Project/predict'))

    for i, filename in enumerate(result_files):
        image_path = os.path.join('/kaggle/working/Project/predict', filename)
        result = detect_foul(image_path, foul_model)

        if result == "foul":
            foul_counter += 1
            shutil.move(image_path, os.path.join(output_folder_foul, f'image_{i}.jpg'))
        else:
            shutil.move(image_path, os.path.join(output_folder_non_foul, f'image_{i}.jpg'))

    return foul_counter

In [14]:
def main(input_video, foul_model):
    frame_array = video2framesarray(input_video)
    temp_images_folder = save_frames_as_images(frame_array)
    temp_images_folder= '/kaggle/working/temp_images'
    yolo_output_images = run_yolo(temp_images_folder)
    total_fouls = process_images(foul_model)
    total_frames = len(yolo_output_images)
    foul_detection_rate = total_fouls / total_frames
    print(f"Foul Detection Rate: {foul_detection_rate}")

if __name__ == '__main__':
    input_video = '/kaggle/input/footballtest/football_test.mp4'
    foul_model_path = '/kaggle/input/models/EfficientB0.h5'
    foul_model = load_model(foul_model_path)
    main(input_video, foul_model)


image 1/318 /kaggle/working/temp_images/image_0.jpg: 640x384 2 persons, 87.1ms
image 2/318 /kaggle/working/temp_images/image_1.jpg: 640x384 2 persons, 78.4ms
image 3/318 /kaggle/working/temp_images/image_10.jpg: 640x384 3 persons, 78.1ms
image 4/318 /kaggle/working/temp_images/image_100.jpg: 640x384 2 persons, 76.3ms
image 5/318 /kaggle/working/temp_images/image_101.jpg: 640x384 2 persons, 77.2ms
image 6/318 /kaggle/working/temp_images/image_102.jpg: 640x384 2 persons, 76.4ms
image 7/318 /kaggle/working/temp_images/image_103.jpg: 640x384 2 persons, 76.5ms
image 8/318 /kaggle/working/temp_images/image_104.jpg: 640x384 2 persons, 76.6ms
image 9/318 /kaggle/working/temp_images/image_105.jpg: 640x384 2 persons, 78.8ms
image 10/318 /kaggle/working/temp_images/image_106.jpg: 640x384 2 persons, 78.1ms
image 11/318 /kaggle/working/temp_images/image_107.jpg: 640x384 2 persons, 76.8ms
image 12/318 /kaggle/working/temp_images/image_108.jpg: 640x384 2 persons, 78.1ms
image 13/318 /kaggle/working/